<a href="https://colab.research.google.com/github/ruheyun/python_pytorch/blob/main/RNN_base_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

词性判断rnn神经网络，pytorch实现

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
training_data = [
    ('the cat ate the fish'.split(), ['det', 'nn', 'v', 'det', 'nn']),
    ('they read that book'.split(), ['nn', 'v', 'det', 'nn'])
]

testing_data = [
    ('they ate the fish'.split())
]

In [ ]:
word_to_idx = {}
tag_to_idx = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)
    for tag in tags:
        if tag not in tag_to_idx:
            tag_to_idx[tag] = len(tag_to_idx)
print(word_to_idx)
print(tag_to_idx)

{'the': 0, 'cat': 1, 'ate': 2, 'fish': 3, 'they': 4, 'read': 5, 'that': 6, 'book': 7}
{'det': 0, 'nn': 1, 'v': 2}


In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_dim), torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return tensor

In [ ]:
embedding_dim = 10
hidden_dim = 3
model = LSTMTagger(embedding_dim, hidden_dim, len(word_to_idx), len(tag_to_idx))
loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

In [ ]:
inputs = prepare_sequence(training_data[0][0], word_to_idx)
tag_scores = model(inputs)
print(training_data[0][0])
print(inputs)
print(tag_scores)
print(torch.max(tag_scores, 1))

['the', 'cat', 'ate', 'the', 'fish']
tensor([0, 1, 2, 0, 3])
tensor([[-1.4063, -1.4401, -0.6577],
        [-1.3587, -1.4525, -0.6752],
        [-1.3959, -1.5474, -0.6170],
        [-1.4101, -1.4654, -0.6446],
        [-1.4283, -1.4377, -0.6485]], grad_fn=<LogSoftmaxBackward0>)
torch.return_types.max(
values=tensor([-0.6577, -0.6752, -0.6170, -0.6446, -0.6485], grad_fn=<MaxBackward0>),
indices=tensor([2, 2, 2, 2, 2]))


In [ ]:
for epoch in range(100):
    model.train()
    for sentence, tags in training_data:
        model.hidden = model.init_hidden()
        sentence_in = prepare_sequence(sentence, word_to_idx)
        targets = prepare_sequence(tags, tag_to_idx)
        tag_scores = model(sentence_in)
        loss = loss_function(tag_scores, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


inputs = prepare_sequence(training_data[0][0], word_to_idx)
tag_scores = model(inputs)
print(training_data[0][0])
print(inputs)
print(tag_scores)
print(torch.max(tag_scores, 1))

['the', 'cat', 'ate', 'the', 'fish']
tensor([0, 1, 2, 0, 3])
tensor([[-1.3316e-01, -2.1069e+00, -5.7890e+00],
        [-8.3180e+00, -7.6003e-04, -7.5700e+00],
        [-5.0714e+00, -5.1951e+00, -1.1887e-02],
        [-3.3127e-03, -6.0865e+00, -6.8743e+00],
        [-5.0985e+00, -1.2095e-02, -5.1301e+00]],
       grad_fn=<LogSoftmaxBackward0>)
torch.return_types.max(
values=tensor([-0.1332, -0.0008, -0.0119, -0.0033, -0.0121], grad_fn=<MaxBackward0>),
indices=tensor([0, 1, 2, 0, 1]))


In [ ]:
test_inputs = prepare_sequence(testing_data[0], word_to_idx)
tag_scores01 = model(test_inputs)
print(testing_data[0])
print(test_inputs)
print(tag_scores01)
print(torch.max(tag_scores01, 1))

['they', 'ate', 'the', 'fish']
tensor([4, 2, 0, 3])
tensor([[-7.7705e+00, -1.8340e-02, -4.0313e+00],
        [-4.5646e+00, -4.9123e+00, -1.7929e-02],
        [-3.1276e-03, -6.2567e+00, -6.7212e+00],
        [-4.9368e+00, -1.4528e-02, -4.9274e+00]],
       grad_fn=<LogSoftmaxBackward0>)
torch.return_types.max(
values=tensor([-0.0183, -0.0179, -0.0031, -0.0145], grad_fn=<MaxBackward0>),
indices=tensor([1, 2, 0, 1]))
